<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14`

`Enter ".help" for usage hints`

`sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);`

`insert into tbl1 values('hello!',10);`

`insert into tbl1 values('goodbye', 20);`

`select * from tbl1;`

Output:  

`hello!|10`

`goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:
  
`.backup ?DB? FILE      Backup DB (default "main") to FILE`

`.cd DIRECTORY          Change the working directory to DIRECTORY`

`.clone NEWDB           Clone data into NEWDB from the existing database`

`.databases             List names and files of attached databases`

`.dump ?TABLE? ...      Dump the database in an SQL text format`

`.excel                 Display the output of next command in a spreadsheet`

`.exit                  Exit this program`

`.headers on|off        Turn display of headers on or off`

`.help                  Show this message`

`.import FILE TABLE     Import data from FILE into TABLE`

`.indexes ?TABLE?       Show names of all indexes`

`.quit                  Exit this program`

`.read FILENAME         Execute SQL in FILENAME`

`.restore ?DB? FILE     Restore content of DB (default "main") from FILE`

`.save FILE             Write in-memory database into FILE`

`.tables ?TABLE?        List names of tables`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [13]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor() 

#.cursor() method is useful when one has multiple connections (and to shorten code)
# c1 = conn.cursor()
# c2 = db_connection.cursor()
# ...

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [3]:
# Create table and columns
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# Save changes
conn.commit()

In [5]:
# Query db (Note! No data has been added yet)
c.execute('SELECT * FROM houses;').fetchall()

[]

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [6]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

In [7]:
c.execute('SELECT * FROM houses;').fetchall()

[(1, 4000, 5, 22, 619000)]

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [8]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 
c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

In [9]:
c.execute('SELECT * FROM houses;').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]

Select all rows from houses

In [10]:
# A
query = 'Select * from houses'
c.execute(query)
c.fetchall()
#pd.DataFrame(c.fetchall())

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [6]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('../../DATA/housing-data.csv', dtype = 'i8', delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [8]:
print(len(data))
display(data[:5])

47


[[2104, 3, 70, 399900],
 [1600, 3, 28, 329900],
 [2400, 3, 44, 369000],
 [1416, 2, 49, 232000],
 [3000, 4, 75, 539900]]

In [9]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

In [10]:
display(data[:5])

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000],
 [None, 1416, 2, 49, 232000],
 [None, 3000, 4, 75, 539900]]

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [11]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [14]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [17]:
c.execute("SELECT * FROM houses;").fetchall()[:5]

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900)]

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [18]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

In [22]:
c.execute("SELECT * FROM houses;").fetchall()[:5]

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900)]

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [20]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do? Delete rows w/ primary key entries 52 and 53

In [24]:
c.execute("SELECT * FROM houses;").fetchall()[45:56]

[(46, 2238, 3, 37, 329900),
 (47, 2567, 4, 57, 314000),
 (48, 1200, 3, 76, 299000),
 (49, 852, 2, 70, 179900),
 (50, 1852, 4, 64, 299900),
 (51, 1203, 3, 11, 239500),
 (54, 2400, 3, 44, 369000),
 (55, 1416, 2, 49, 232000),
 (56, 3000, 4, 75, 539900),
 (57, 1985, 4, 61, 299900),
 (58, 1534, 3, 12, 314900)]

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [22]:
# ANSWER
query = """SELECT * FROM houses WHERE bdrms = 4;"""
c.execute(query)
c.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. The following query calculates the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [23]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [25]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [26]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('../../DATA/housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [27]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

47

Run a query to get the average price of each house size from this table:

In [33]:
#ANSWER
query = 'SELECT AVG(price), bdrms FROM houses_pandas GROUP BY bdrms'
pd.read_sql(query, conn)
# or
# c.execute(query)
# c.fetchall()

,AVG(price),bdrms
0,169900.000000,1
1,280866.666667,2
2,326403.920000,3
3,377449.785714,4
4,699900.000000,5


## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database? For data cleaning

2. When would you want to use Pandas on a dataset retrieved from a database? For data visualisation

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [28]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [29]:
# Reading CSV to Dataframe
orders = pd.read_csv('../../DATA/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('../../DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [30]:
orders_break_down.shape

(8047, 6)

In [31]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [32]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [33]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [34]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [35]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [36]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [37]:
# Establishing Local DB connection
db_connection = sqlite3.connect('../../DATA/eshop.db.sqlite')

# Save these two dataframes as a table in sqlite
orders.to_sql('orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down', con = db_connection, if_exists = 'replace', index = False)

8047

In [39]:
db_connection.execute("SELECT * FROM orders;").fetchall()[:5]

[('IT-2011-3647632', '2011-01-01', 'Eugene Moren', 'Sweden'),
 ('ES-2011-4869686', '2011-01-03', 'Dorothy Dickinson', 'United Kingdom'),
 ('ES-2011-4939443', '2011-01-04', 'Arthur Prichep', 'France'),
 ('IT-2011-2942451', '2011-01-04', 'Grant Thornton', 'United Kingdom'),
 ('ES-2011-3848439', '2011-01-05', 'Michael Granlund', 'France')]

In [41]:
# equals to
c = db_connection.cursor()
c.execute("SELECT * FROM orders;").fetchall()[:5]

[('IT-2011-3647632', '2011-01-01', 'Eugene Moren', 'Sweden'),
 ('ES-2011-4869686', '2011-01-03', 'Dorothy Dickinson', 'United Kingdom'),
 ('ES-2011-4939443', '2011-01-04', 'Arthur Prichep', 'France'),
 ('IT-2011-2942451', '2011-01-04', 'Grant Thornton', 'United Kingdom'),
 ('ES-2011-3848439', '2011-01-05', 'Michael Granlund', 'France')]

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [59]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [79]:
# Select Number of Distinct Orders for Each Customer

query = """
        SELECT COUNT (DISTINCT order_id) AS order_count, customer_name
        FROM orders
        GROUP BY customer_name
        ORDER BY order_count DESC;
        """

distinct_orders = sql.read_sql(query, con=db_connection)
distinct_orders.head()

,order_count,customer_name
0,13,John Grady
1,12,Joel Jenkins
2,12,Aaron Smayling
3,11,Yoseph Carroll
4,11,Sarah Brown


#### 6. Select Number of Customers for Each Country

In [78]:
# Select Number of Customers for Each Country
sql.read_sql("""SELECT COUNT(customer_name) AS customer_count, country
                FROM orders
                GROUP BY country
                ORDER BY customer_count DESC
                LIMIT 5
                ;""",
            db_connection)

,customer_count,country
0,991,France
1,806,Germany
2,700,United Kingdom
3,493,Italy
4,403,Spain


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [74]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
sql.read_sql("""SELECT order_id, discount, sales, quantity
                FROM orders_break_down
                LIMIT 5
                ;""",
            db_connection)

,order_id,discount,sales,quantity
0,IT-2011-3647632,0.5,45.0,3
1,ES-2011-4869686,0.0,854.0,7
2,ES-2011-4939443,0.0,140.0,3
3,IT-2011-2942451,0.5,27.0,2
4,IT-2011-2942451,0.5,17.0,2


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [73]:
# Select discount, sales, quantity, total price for Each Order Id from orders_break_down Table
sql.read_sql("""SELECT order_id, discount, sales, quantity, (sales * quantity - discount) AS total_price
                FROM orders_break_down
                ORDER BY total_price DESC
                LIMIT 5
                ;""",
            db_connection)

,order_id,discount,sales,quantity,total_price
0,ES-2014-3785216,0.1,5729.0,14,80205.9
1,ES-2011-3248922,0.0,6517.0,12,78204.0
2,ES-2012-2183106,0.1,4748.0,14,66471.9
3,IT-2013-4602742,0.1,5726.0,10,57259.9
4,ES-2014-2637201,0.0,4141.0,13,53833.0


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [72]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
sql.read_sql("""SELECT *, (sales * quantity - discount) AS total_price
                FROM orders_break_down
                WHERE total_price > 100
                ORDER BY total_price ASC
                LIMIT 5
                ;""",
            db_connection)

,order_id,product_name,discount,sales,quantity,category,total_price
0,IT-2011-2668777,"Binney & Smith Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
1,IT-2011-1165265,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5
2,ES-2012-1763830,"Elite Scissors, High Speed",0.5,34.0,3,Office Supplies,101.5
3,IT-2012-4099397,"Sanford Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
4,ES-2013-4250046,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5


#### 8. Select All Customers And The Products They Have Bought

In [67]:
# Select All Order Ids, Customers And The Products (product_name) They Have Bought
sql.read_sql("""SELECT orders.order_id, customer_name, product_name
                FROM orders
                JOIN orders_break_down 
                ON orders.order_id = orders_break_down.order_id
                LIMIT 5
                ;""",
            db_connection)

,order_id,customer_name,product_name
0,IT-2011-3647632,Eugene Moren,"Enermax Note Cards, Premium"
1,ES-2011-4869686,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,ES-2011-4939443,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,IT-2011-2942451,Grant Thornton,"Boston Markers, Easy-Erase"
4,IT-2011-2942451,Grant Thornton,"Eldon Folders, Single Width"


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [66]:
# Select Number of 'Furniture' Orders For Each Country
sql.read_sql("""SELECT COUNT(category) AS order_quantity, country
                FROM orders
                JOIN orders_break_down 
                ON orders.order_id = orders_break_down.order_id
                WHERE category = 'Furniture'
                GROUP BY country
                LIMIT 5
                ;""",
            db_connection)

,order_quantity,country
0,40,Austria
1,20,Belgium
2,9,Denmark
3,16,Finland
4,299,France


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [65]:
# Select Number of 'Furniture' Orders For The Country Denmark
sql.read_sql("""SELECT *
                FROM orders
                JOIN orders_break_down 
                ON orders.order_id = orders_break_down.order_id
                WHERE category = 'Furniture'
                AND country = 'Denmark'
                LIMIT 5
                ;""",
            db_connection)

,order_id,order_date,customer_name,country,order_id,product_name,discount,sales,quantity,category
0,ES-2011-1586186,2011-06-16,Vivek Grady,Denmark,ES-2011-1586186,"Office Star Bag Chairs, Set of Two",0.5,87.0,3,Furniture
1,IT-2012-4084224,2012-02-15,Mick Brown,Denmark,IT-2012-4084224,"Rubbermaid Door Stop, Ergonomic",0.6,17.0,1,Furniture
2,IT-2013-4649004,2013-05-31,Bart Folk,Denmark,IT-2013-4649004,"Novimex Executive Leather Armchair, Red",0.5,920.0,4,Furniture
3,ES-2013-3422294,2013-09-07,Dave Brooks,Denmark,ES-2013-3422294,"Advantus Door Stop, Duo Pack",0.6,36.0,2,Furniture
4,ES-2014-3653593,2014-02-10,Ivan Liston,Denmark,ES-2014-3653593,"Deflect-O Clock, Black",0.6,161.0,8,Furniture


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [64]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country
sql.read_sql("""SELECT country,
                SUM(CASE WHEN discount > 0 THEN sales ELSE 0 END) AS sales_with_discount,
                SUM(CASE WHEN discount = 0 THEN sales ELSE 0 END) AS sales_without_discount
                FROM orders
                JOIN orders_break_down
                ON orders.order_id = orders_break_down.order_id
                GROUP BY country
                LIMIT 5
                ;""",
            db_connection)

,country,sales_with_discount,sales_without_discount
0,Austria,0.0,79382.0
1,Belgium,0.0,42320.0
2,Denmark,7763.0,0.0
3,Finland,0.0,20702.0
4,France,479892.0,129791.0


#### 11.A Select Total Quantity, Total Sales for Each Country

In [63]:
# Select Total Quantity, Total Sales for Each Country
sql.read_sql("""SELECT SUM(sales) AS total_sales, SUM(quantity) AS total_quantity, country
                FROM orders
                JOIN orders_break_down 
                ON orders.order_id = orders_break_down.order_id
                GROUP BY country
                LIMIT 5
                ;""",
            db_connection)

,total_sales,total_quantity,country
0,79382.0,973,Austria
1,42320.0,532,Belgium
2,7763.0,204,Denmark
3,20702.0,201,Finland
4,609683.0,7329,France


#### 11.B Select Top 3 Countries and Quantities Based on Total Sales

In [61]:
# Select Top 3 Countries and Quantities Based on Total Sales
sql.read_sql("""SELECT SUM(sales) AS total_sales, SUM(quantity) AS total_quantity, country
                FROM orders
                JOIN orders_break_down 
                ON orders.order_id = orders_break_down.order_id
                GROUP BY country
                ORDER BY total_sales DESC
                LIMIT 3
                ;""",
            db_connection)

,total_sales,total_quantity,country
0,609683.0,7329,France
1,488681.0,6179,Germany
2,420497.0,4917,United Kingdom


#### 11.C Select Bottom 3 Countries and Sales Based On Total Quantities

In [60]:
# Select Bottom 3 Countries and Sales Based On Total Quantities
sql.read_sql("""SELECT SUM(sales) AS total_sales, SUM(quantity) AS total_quantity, country
                FROM orders
                JOIN orders_break_down 
                ON orders.order_id = orders_break_down.order_id
                GROUP BY country
                ORDER BY total_quantity
                LIMIT 3
                ;""",
            db_connection)

,total_sales,total_quantity,country
0,20702.0,201,Finland
1,7763.0,204,Denmark
2,20529.0,261,Norway


#### 12. Select Average Sales By Category For The Country 'France'

In [59]:
# Select Average Sales By Category For The Country 'France'
sql.read_sql("""SELECT AVG(sales) AS avg_sales, category
                FROM orders
                JOIN orders_break_down 
                ON orders.order_id = orders_break_down.order_id
                WHERE country = 'France'
                GROUP BY category
                ;""",
            db_connection)

,avg_sales,category
0,582.314381,Furniture
1,167.217709,Office Supplies
2,595.145078,Technology


#### 13. Select Country, Category and Average Sales Where Average Sales is The Highest

In [58]:
# Select Country, Category and Average Sales Where Average Sales is The Highest
sql.read_sql("""SELECT AVG(sales) AS avg_sales, category, country
                FROM orders
                JOIN orders_break_down
                ON orders.order_id = orders_break_down.order_id
                GROUP BY category, country
                ORDER BY avg_sales DESC
                LIMIT 1
                ;""",
            db_connection)

,avg_sales,category,country
0,902.846154,Technology,Switzerland


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



